# Exercise 02 : Join

In [1]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## Create a new table datamart in the database by joining the tables pageviews and checker using only one query

- the table should have the following columns: uid, labname, first_commit_ts, first_view_ts
- first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user
- first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed
- status = ’ready’ should still be a filter
- numTrials = 1 should still be a filter
- labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
- the table should contain only the users (uids with user_*) and not the admins
- first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [3]:
datamart_query = """
CREATE TABLE datamart AS
SELECT c.uid, c.labname, MIN(c.timestamp) AS first_commit_ts, MIN(p.datetime) AS first_view_ts
FROM checker c
LEFT JOIN pageviews p ON c.uid = p.uid
WHERE c.status = 'ready' AND c.numTrials = 1 AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname;
"""
conn.execute(datamart_query)

## Using Pandas methods, create two dataframes: test and control

- test should have the users that have the values in first_view_ts
- control should have the users that have missing values in first_view_ts
- replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis
- save both tables into the database, you will use them in the next exercises

In [4]:
# Create test and control dataframes
datamart_df = pd.io.sql.read_sql("SELECT * FROM datamart", conn)
datamart_df['first_commit_ts'] = pd.to_datetime(datamart_df['first_commit_ts'])
datamart_df['first_view_ts'] = pd.to_datetime(datamart_df['first_view_ts'])
test = datamart_df[datamart_df['first_view_ts'].notnull()]
control = datamart_df[datamart_df['first_view_ts'].isnull()]

In [5]:
# Replace missing values in control with average first_view_ts of test users
control_avg = test['first_view_ts'].mean()
control.loc[:, 'first_view_ts'] = control['first_view_ts'].fillna(control_avg)

In [6]:
# Save test and control tables into the database
test.to_sql('test', conn, index=False)
control.to_sql('control', conn, index=False)

81

## Close the connection

In [7]:
conn.close()